# Virtual inertia scheduling (VIS) for IEEE39

In [1]:
from andes.interop.pandapower import to_pandapower
from andes.interop.pandapower import make_GSF, build_group_table
import andes
import gurobipy as gb
import pandas as pd
import numpy as np
import logging
import torch
import os

logger = logging.getLogger(__name__)

from visopf import vis1, vis2, loadnn

## load data

In [2]:
# ----- get andes case from excel ------
dir_path = os.path.abspath('..')
case_path = '/VIS_opf/ieee39_vis.xlsx'
case = dir_path + case_path
ssa = andes.load(case, no_output=True)

In [3]:
# ----- get neural network data ------
nn_path = '/VIS_opf/NN_train'
nn, norm = loadnn(nn_path)

## build optimization model and solve

In [4]:
# ----- set sim scenario -----
ss = vis2(norm=norm, nn=nn, dpe=-0.1, rocof_lim=0.033, nadir_lim=0.01)

# ----- set vsg gen -----
vsg_ieee39 = ['PV_1', 'PV_6', 'PV_8', 'PV_9']
ss.from_andes(ssa, vsg_ieee39, Sbase=1000)

# ------ set gen cost ------
# iloc[0] - iloc[9]
# iloc[9]: slack bus
# iloc[0],[5],[8],[9]: vsg
ss.cost['c1'].iloc[0] = 1
ss.cost['c1'].iloc[5] = 0.1
ss.cost['c1'].iloc[9] = 1.5
ss.update_dict()

# ss.costdict

Restricted license - for non-production use only - expires 2023-10-25


Note: \ Control (dynamic) parameters are renormalized based on case Sbase rather then to andes base


In [5]:
ss.build()

Successfully build var.
Successfully build obj.
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Successfully build cons.


In [6]:
ss.get_res()

Successfully build var.
Successfully build obj.
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Successfully build cons.
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1040 rows, 678 columns and 11758 nonzeros
Model fingerprint: 0x5914150c
Variable types: 358 continuous, 320 integer (320 binary)
Coefficient statistics:
  Matrix range     [2e-07, 1e+02]
  Objective range  [1e-01, 2e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [2e-03, 2e+03]
Found heuristic solution: objective 52.8640000
Presolve removed 553 rows and 338 columns
Presolve time: 0.03s
Presolved: 487 rows, 340 columns, 5740 nonzeros
Variable types: 184 continuous, 156 integer (156 binary)

Root relaxation: cutoff, 0 iterations, 0.01 seconds (0.00 work units)

Explored 1 nodes (0 simplex iterations) in 0.06 seconds (0.03 work units)
Thread count was 8 (of 8 available proces

Msys and Dsys are normlized by devise Sbase, transform to andes Sbase when do TDS


(        gen      Sn      pg  pru       prd
 0      PV_1  10.400   8.000  0.0  0.026380
 1      PV_2   8.360   9.000  0.0  0.000000
 2      PV_3   8.437   8.000  0.0  0.000000
 3      PV_4  11.748   3.564  0.0  0.000000
 4      PV_5  10.802   1.000  0.0  0.000000
 5      PV_6  10.857   8.000  0.0  0.056832
 6      PV_7  10.252   1.000  0.0  0.000000
 7      PV_8   9.702   7.000  0.0  0.005843
 8      PV_9  16.841  10.000  0.0  0.005843
 9  Slack_10  11.990   3.000  0.0  0.000000,
     gen      Mvsg  Dvsg  pg_vsg  pru_vsg   prd_vsg  pmax_vsg  pmin_vsg
 0  PV_1  1.801179   0.0     8.0      0.0  0.026380      15.0       0.0
 1  PV_6  5.000000   0.0     8.0      0.0  0.056832       8.0       0.0
 2  PV_8  0.000000   0.0     7.0      0.0  0.005843       7.0       0.0
 3  PV_9  0.000000   0.0    10.0      0.0  0.005843      10.0       0.0,
 {'Msys': 5.413029429232188,
  'Dsys': 0.5893050315845287,
  'Rsys': 20.933442043222,
  'Fsys': 7.846526879799961})